In [12]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client
import pandas as pd
from glob import glob as gb

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
import gsw
session = cc.database.create_session()

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:42387 Dashboard: http://127.0.0.1:34688/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
input_path = '/g/data/e14/erd561/ARGO/'

output_path = '/g/data/e14/erd561/Australia_2004-2017/'

In [8]:
lonW = 88
lonE = 182

latS = -72
latN = 22

temp_z0_LTM = xr.open_dataset(input_path + 'RG_ArgoClim_Temperature_2017.nc', decode_times=False).\
                ARGO_TEMPERATURE_MEAN.\
                sel(LONGITUDE=slice(lonW,lonE)).\
                sel(LATITUDE=slice(latS,latN)).\
                sel(PRESSURE=0, method='nearest')
print(temp_z0_LTM)

<xarray.DataArray 'ARGO_TEMPERATURE_MEAN' (LATITUDE: 87, LONGITUDE: 94)>
array([[-1.447, -1.462, -1.486, ..., -0.234, -0.219, -0.243],
       [-1.222, -1.215, -1.221, ...,  0.571,  0.524,  0.466],
       [-1.   , -0.969, -0.946, ...,  1.427,  1.343,  1.259],
       ...,
       [28.252, 28.196, 28.066, ..., 27.148, 27.138, 27.091],
       [28.115, 28.022,    nan, ..., 26.993, 26.986, 26.971],
       [   nan,    nan,    nan, ..., 26.653, 26.65 , 26.669]], dtype=float32)
Coordinates:
  * LONGITUDE  (LONGITUDE) float32 88.5 89.5 90.5 91.5 ... 179.5 180.5 181.5
  * LATITUDE   (LATITUDE) float32 -64.5 -63.5 -62.5 -61.5 ... 19.5 20.5 21.5
    PRESSURE   float32 2.5
Attributes:
    units:      degree celcius (ITS-90)
    long_name:  ARGO TEMPERATURE MEAN Jan 2004 - Dec 2016 (13.0 year) RG CLIM...


In [9]:
temp_z0_months = xr.open_dataset(input_path + 'RG_ArgoClim_Temperature_2017.nc', decode_times=False).\
                ARGO_TEMPERATURE_ANOMALY.\
                sel(LONGITUDE=slice(lonW,lonE)).\
                sel(LATITUDE=slice(latS,latN)).\
                sel(PRESSURE=0, method='nearest')
print(temp_z0_months)

TIME = [pd.date_range('15/1/2004',periods=156, freq=pd.DateOffset(months=1))]
print(TIME)

temp_z0_months = temp_z0_months.assign_coords(TIME=np.squeeze(TIME))
print(temp_z0_months)

temp_z0 = temp_z0_months.groupby('TIME.year').mean('TIME', skipna=True)
print(temp_z0)

temp_z0_2004_2016 = temp_z0.sel(year=slice(1993,2017))
print(temp_z0_2004_2016)

<xarray.DataArray 'ARGO_TEMPERATURE_ANOMALY' (TIME: 156, LATITUDE: 87, LONGITUDE: 94)>
[1275768 values with dtype=float32]
Coordinates:
  * LONGITUDE  (LONGITUDE) float32 88.5 89.5 90.5 91.5 ... 179.5 180.5 181.5
  * LATITUDE   (LATITUDE) float32 -64.5 -63.5 -62.5 -61.5 ... 19.5 20.5 21.5
    PRESSURE   float32 2.5
  * TIME       (TIME) float32 0.5 1.5 2.5 3.5 4.5 ... 152.5 153.5 154.5 155.5
Attributes:
    units:      degree celcius (ITS-90)
    long_name:  ARGO TEMPERATURE ANOMALY defined by Jan 2004 - Dec 2016 (13.0...
[DatetimeIndex(['2004-01-15', '2004-02-15', '2004-03-15', '2004-04-15',
               '2004-05-15', '2004-06-15', '2004-07-15', '2004-08-15',
               '2004-09-15', '2004-10-15',
               ...
               '2016-03-15', '2016-04-15', '2016-05-15', '2016-06-15',
               '2016-07-15', '2016-08-15', '2016-09-15', '2016-10-15',
               '2016-11-15', '2016-12-15'],
              dtype='datetime64[ns]', length=156, freq='<DateOffset: months=1>')]

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'ARGO_TEMPERATURE_ANOMALY' (year: 13, LATITUDE: 87, LONGITUDE: 94)>
array([[[-0.00483336, -0.00474999,  0.00108333, ...,  0.02583335,
          0.02641667,  0.02499998],
        [ 0.00616669, -0.00641666, -0.01150001, ...,  0.01424997,
          0.02108336,  0.02183335],
        [ 0.03125001,  0.01600003,  0.00575001, ...,  0.00366667,
          0.00716668,  0.01041667],
        ...,
        [-0.01458337, -0.02033331, -0.01600001, ...,  0.03008332,
          0.03958337,  0.04891667],
        [-0.00533332, -0.00625   ,         nan, ...,  0.00816671,
          0.01783331,  0.02391663],
        [        nan,         nan,         nan, ..., -0.01400002,
         -0.00650002, -0.00050006]],

       [[-0.02024999, -0.01658333, -0.00716668, ...,  0.12591667,
          0.12366664,  0.1195    ],
        [-0.01616665, -0.02466667, -0.02591664, ...,  0.12049999,
          0.12433332,  0.12158332],
        [ 0.00566668, -0.00483334, -0.01233332, ...,  0.11625003,
          0.11641

In [10]:
temp_z0_2017 = xr.open_mfdataset(sorted(gb(input_path + 'RG_ArgoClim_2017*.nc')), decode_times=False).\
                ARGO_TEMPERATURE_ANOMALY.\
                sel(LONGITUDE=slice(lonW,lonE)).\
                sel(LATITUDE=slice(latS,latN)).\
                sel(PRESSURE=0, method='nearest').\
                assign_coords(TIME=np.squeeze([pd.date_range('15/1/2017',periods=12, freq=pd.DateOffset(months=1))])).\
                groupby('TIME.year').mean('TIME', skipna=True)
temp_z0_2017.load()
print(temp_z0_2017)

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  """Entry point for launching an IPython kernel.
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


<xarray.DataArray 'ARGO_TEMPERATURE_ANOMALY' (year: 1, LATITUDE: 87, LONGITUDE: 94)>
array([[[0.2570833 , 0.22524999, 0.2065833 , ..., 0.59825   ,
         0.59675   , 0.5605833 ],
        [0.23733337, 0.21266662, 0.23291664, ..., 0.7073334 ,
         0.72166663, 0.72374994],
        [0.23558332, 0.22275   , 0.24216668, ..., 0.54983336,
         0.6028333 , 0.65825003],
        ...,
        [0.18266664, 0.14475   , 0.1541667 , ..., 0.46650004,
         0.48808336, 0.41766658],
        [0.08566663, 0.08975001,        nan, ..., 0.32199997,
         0.4095    , 0.34424996],
        [       nan,        nan,        nan, ..., 0.24091668,
         0.40475   , 0.39575002]]], dtype=float32)
Coordinates:
  * LATITUDE   (LATITUDE) float32 -64.5 -63.5 -62.5 -61.5 ... 19.5 20.5 21.5
  * LONGITUDE  (LONGITUDE) float32 88.5 89.5 90.5 91.5 ... 179.5 180.5 181.5
    PRESSURE   float32 2.5
  * year       (year) int64 2017


In [13]:
temp_z0 = xr.concat([temp_z0_2004_2016, temp_z0_2017], dim='year') + temp_z0_LTM
print(temp_z0)

# temp_z0 = temp_z0.assign_coords(PRESSURE=gsw.z_from_p(temp_z0.PRESSURE, temp_z0.LATITUDE))
# print(temp_z0)

<xarray.DataArray (year: 14, LATITUDE: 87, LONGITUDE: 94)>
array([[[-1.4518334e+00, -1.4667500e+00, -1.4849166e+00, ...,
         -2.0816664e-01, -1.9258332e-01, -2.1800002e-01],
        [-1.2158333e+00, -1.2214167e+00, -1.2325000e+00, ...,
          5.8524996e-01,  5.4508334e-01,  4.8783335e-01],
        [-9.6875000e-01, -9.5299995e-01, -9.4024998e-01, ...,
          1.4306667e+00,  1.3501668e+00,  1.2694166e+00],
        ...,
        [ 2.8237417e+01,  2.8175665e+01,  2.8049999e+01, ...,
          2.7178083e+01,  2.7177584e+01,  2.7139915e+01],
        [ 2.8109667e+01,  2.8015749e+01,            nan, ...,
          2.7001167e+01,  2.7003834e+01,  2.6994917e+01],
        [           nan,            nan,            nan, ...,
          2.6639000e+01,  2.6643499e+01,  2.6668501e+01]],

       [[-1.4672500e+00, -1.4785833e+00, -1.4931666e+00, ...,
         -1.0808332e-01, -9.5333353e-02, -1.2350000e-01],
        [-1.2381667e+00, -1.2396667e+00, -1.2469167e+00, ...,
          6.9149995e-01,

ValueError: cannot add coordinates with new dimensions to a DataArray

In [17]:
year = temp_z0.year
lat = temp_z0.LATITUDE
lon = temp_z0.LONGITUDE

# temp_z0_trans = np.transpose(temp_z0, (2,1,0))
temp_z0_trans = temp_z0
print(np.shape(temp_z0_trans))
temp_z0_slope = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'depth', 'lon'), coords=[lat, lon], name='temp_z0')
temp_z0_p_value = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'depth', 'lon'), coords=[lat, lon], name='temp_z0')
temp_z0_std_err = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'depth', 'lon'), coords=[lat, lon], name='temp_z0')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):
        temp_z0_slope[iid, jid], _, _, temp_z0_p_value[iid, jid], temp_z0_std_err[iid, jid] = \
        stats.linregress(year, temp_z0_trans[:, iid, jid])
    print('depth ' + str(np.array(i)))
print(temp_z0_slope)

(14, 87, 94)
depth -64.5
depth -63.5
depth -62.5
depth -61.5
depth -60.5
depth -59.5
depth -58.5
depth -57.5
depth -56.5
depth -55.5
depth -54.5
depth -53.5
depth -52.5
depth -51.5
depth -50.5
depth -49.5
depth -48.5
depth -47.5
depth -46.5
depth -45.5
depth -44.5
depth -43.5
depth -42.5
depth -41.5
depth -40.5
depth -39.5
depth -38.5
depth -37.5
depth -36.5
depth -35.5
depth -34.5
depth -33.5
depth -32.5
depth -31.5
depth -30.5
depth -29.5
depth -28.5
depth -27.5
depth -26.5
depth -25.5
depth -24.5
depth -23.5
depth -22.5
depth -21.5
depth -20.5
depth -19.5
depth -18.5
depth -17.5
depth -16.5
depth -15.5
depth -14.5
depth -13.5
depth -12.5
depth -11.5
depth -10.5
depth -9.5
depth -8.5
depth -7.5
depth -6.5
depth -5.5
depth -4.5
depth -3.5
depth -2.5
depth -1.5
depth -0.5
depth 0.5
depth 1.5
depth 2.5
depth 3.5
depth 4.5
depth 5.5
depth 6.5
depth 7.5
depth 8.5
depth 9.5
depth 10.5
depth 11.5
depth 12.5
depth 13.5
depth 14.5
depth 15.5
depth 16.5
depth 17.5
depth 18.5
depth 19.5
depth 2

In [18]:
temp_z0_xr = xr.DataArray(np.transpose(temp_z0.values, (1,2,0)), name='temp_z0_argo',
                        coords=[lat, lon, year], 
                        dims=['lat', 'lon', 'year'])
print(temp_z0_xr)
temp_z0_xr.to_netcdf(output_path + 'temp_z0_argo.nc')

temp_z0_slope_xr = xr.DataArray(temp_z0_slope, name='temp_z0_slope_argo',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(temp_z0_slope_xr)
temp_z0_slope_xr.to_netcdf(output_path + 'temp_z0_slope_argo.nc')

temp_z0_p_value_xr = xr.DataArray(temp_z0_p_value, name='temp_z0_p_value_argo',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(temp_z0_p_value_xr)
temp_z0_p_value_xr.to_netcdf(output_path + 'temp_z0_p_value_argo.nc')

temp_z0_std_err_xr = xr.DataArray(temp_z0_std_err, name='temp_z0_std_err_argo',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(temp_z0_std_err_xr)
temp_z0_std_err_xr.to_netcdf(output_path + 'temp_z0_std_err_argo.nc')

<xarray.DataArray 'temp_z0_argo' (lat: 87, lon: 94, year: 14)>
array([[[-1.4518334e+00, -1.4672500e+00, -1.5691667e+00, ...,
         -1.5306667e+00, -1.3813334e+00, -1.1899167e+00],
        [-1.4667500e+00, -1.4785833e+00, -1.5815833e+00, ...,
         -1.5024166e+00, -1.4068334e+00, -1.2367500e+00],
        [-1.4849166e+00, -1.4931666e+00, -1.5970833e+00, ...,
         -1.4998332e+00, -1.4564999e+00, -1.2794167e+00],
        ...,
        [-2.0816664e-01, -1.0808332e-01, -3.3333227e-02, ...,
         -3.2408333e-01, -2.1041666e-01,  3.6424997e-01],
        [-1.9258332e-01, -9.5333353e-02, -2.0083308e-02, ...,
         -2.7583334e-01, -1.4166667e-01,  3.7775004e-01],
        [-2.1800002e-01, -1.2350000e-01, -4.8250005e-02, ...,
         -2.4683338e-01, -1.3449997e-01,  3.1758329e-01]],

       [[-1.2158333e+00, -1.2381667e+00, -1.3470833e+00, ...,
         -1.3940833e+00, -1.2208333e+00, -9.8466665e-01],
        [-1.2214167e+00, -1.2396667e+00, -1.3503333e+00, ...,
         -1.3361667e